In [1]:
from bs4 import BeautifulSoup
import lxml
import re
import os.path
my_path = os.path.abspath('')

In [2]:
#to be implemented
def check_parsing_error(file):
    for section in sections:
        if not(test_str2 and test_str2.strip()):
            print("section is empty")
    return None

In [3]:
#Data cleaning approach:

#First identify the table of content by regex to get mention of "Item" in <Table> <tbody> tag(perhaps extract the table of content)
#or search <a href="#....." util the mention of Item16, signautre
#extract all the href in <a> attribute in the table of content (ex. href1, href2, href3 …)

#Second have algorithm to identify <a name="href1"> and save the content between href1 and href2, href2 and href3, …

#Third, clean the html tags in the contents 

In [4]:
def load_html(path):
    return BeautifulSoup(open(path,"r"), 'html.parser')
def load_xml(path):
    return BeautifulSoup(open(path,"r"), 'lxml')

#get the text from soup
def soup_get_text(soup):
    print(soup.get_text())

#write "string" to file from "path"
def write_file(path, string):
    text_file = open(path, "w")
    text_file.write(string)
    text_file.close()


#print elements from list
def print_list(lists):
    for list in lists:
        print(list)

#read all the text file in the directory
def file_reader(path):
    files = []
    for r,d,f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r,file))
                
    return files


In [5]:
#First extract the location of the table of content by
#using regex to identify "Item 15" or "Financial Statement Schedules"
#get every links <a href="#"> above "Item 15" or "Financial Statement Schedules"
#only works with the text and items in table of content contains href#
#if only in the page number, doesn't work
def get_table_of_content_links(soup):
    tag = soup.find('a',string=re.compile("Financial Statement Schedules"))
    tag_item = soup.find('a',string=re.compile("Item 15"))
    if tag==None and tag_item!=None:
        tag = tag_item
    
    links = list()
    links.append(tag.get('href')[1:])
    for elements in tag.previous_elements:
        if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
            #print(elements)
            links.append(elements.get('href')[1:])
    
    links.reverse()
    print("total section: ",len(remove_duplicates_in_list(links)))
    return remove_duplicates_in_list(links)

#remove dupicated items
def remove_duplicates_in_list(List):
    return list(dict.fromkeys(List))
"""
def get_table_of_content_links2(soup):
    links = list()
    for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
        if link.get('href')!=None and link.get('href')[0]=='#':
            links.append(link.get('href')[1:])
    return links
# or identify the signature link and use find_previous method to extract:extract()?
"""


"\ndef get_table_of_content_links2(soup):\n    links = list()\n    for link in soup.find_all('a'):\n    #link.get('href')\n    #print(link.get('href'))\n    #print(link.get('name'))\n        if link.get('href')!=None and link.get('href')[0]=='#':\n            links.append(link.get('href')[1:])\n    return links\n# or identify the signature link and use find_previous method to extract:extract()?\n"

In [6]:
#get every sections by using the links to identify <a>
def get_all_sections(links, soup):
    sections=list()
    for i in range(len(links)):
        if i<len(links)-1:
            sections.append(get_section(links[i],links[i+1],soup))
        else:
            sections.append(get_last_section(links[i],soup))
    return sections

#get sections between links[i] and links[i+1]
def get_section(link1, link2, soup):
    section_str=""
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('name') and elements['name']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

#get last section that is between <a name="links[n]"> to the end of the document
def get_last_section(link1,soup):
    section_str=""
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

In [7]:
#Remove all the tag that is not <type>10-k using xml
def clean_10k_by_xml(path):
    soup=load_xml(path)
    extract_type_10k(soup)
    write_file(path, str(soup))

#remove all tag that is not <type>10-k
def extract_type_10k(soup):
    tag_list=soup.find_all('type')
    for tag in tag_list:
        print(tag.next_element)
        if '10-K' not in tag.next_element:
            tag.extract()
    

In [8]:
#First remove all the tag that is not <type>10-k including xml
#Secont extract the links of the items from the table of content 
#Third get all the sections and store it in a list
def get_10k_sections(path):
    soup=load_html(path)
    sections=get_all_sections(get_table_of_content_links(load_html(path)),soup)
    return sections

#get all the path of 10k file and store in list name files
def get_10k_files():
    files = [];
    files = file_reader(my_path+"/sec_edgar_filings")
    return files

In [9]:
#print(sections[0])
def save_sections(tenk_filings):
    report_number = 0
    section_num =0
    for report in tenk_filings:
        if not os.path.exists(my_path+'/clean_data/filing_'+str(report_number)):
            os.mkdir(my_path+'/clean_data/filing_'+str(report_number))
            print("Directory " , '/clean_data/filing_'+str(report_number) ,  " Created ")
        else:    
            print("Directory " , '/clean_data/filing_'+str(report_number) ,  " already exists")
        
        for section in report:
            write_file('clean_data/filing_'+str(report_number)+'/filing_'+str(report_number)+'_section_'+str(section_num)+".txt",section)
            section_num = section_num+1
        report_number = report_number+1

def save_sections(filing, path, report_number):
    section_num =0
    for section in filing:
        write_file(path+'/filing_'+str(report_number)+'_section_'+str(section_num)+".txt",section)
        section_num = section_num+1


def save_filings_by_section(filings, path):
    report_number = 0
    if not os.path.exists(path+'/clean_data'):
        os.mkdir(path+'/clean_data')
        print("Directory " , '/clean_data/',  " Created ")
    for filing in tenk_filings:
        if not os.path.exists(path+'/clean_data/filing_'+str(report_number)):
            os.mkdir(path+'/clean_data/filing_'+str(report_number))
            print("Directory " , '/clean_data/filing_'+str(report_number) ,  " Created ")
        else:    
            print("Directory " , '/clean_data/filing_'+str(report_number) ,  " already exists")
        
        
        save_sections(filing,path+'/clean_data/filing_'+str(report_number), report_number)
        report_number = report_number+1



In [10]:
files_path=list()
files_path=get_10k_files()
print_list(files_path)


/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0000320193-19-000119.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0001628280-16-020309.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0000320193-17-000070.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0001193125-15-356351.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0000320193-18-000145.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/GOOG/10-K/0001652044-19-000004.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/GOOG/10-K/0001652044-17-000008.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Da

In [11]:
for file in files_path:
    print(file)
    clean_10k_by_xml(file)


/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0000320193-19-000119.txt
10-K

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0001628280-16-020309.txt
10-K

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0000320193-17-000070.txt
10-K

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0001193125-15-356351.txt
10-K

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AAPL/10-K/0000320193-18-000145.txt
10-K

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/GOOG/10-K/0001652044-19-000004.txt
10-K

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/GOOG/10-K/0001652044-17-000008.txt
10-K

/Users/davidren/Desktop/Education/

In [12]:
sections=list()
tenk_filings = list()
for file in files_path:
    sections=get_10k_sections(file)
    tenk_filings.append(sections)

#save_filings_by_section(tenk_filings, my_path)

total section:  24
total section:  24
total section:  24
total section:  25
total section:  24
total section:  22
total section:  22
total section:  22
total section:  22
total section:  23
total section:  20
total section:  20
total section:  20
total section:  20
total section:  20
total section:  23
total section:  23
total section:  24
total section:  24
total section:  23
total section:  25
total section:  24
total section:  25
total section:  24
total section:  25
total section:  25
total section:  25
total section:  25
total section:  25


In [13]:
save_filings_by_section(tenk_filings, my_path)

Directory  /clean_data/filing_0  already exists
Directory  /clean_data/filing_1  Created 
Directory  /clean_data/filing_2  Created 
Directory  /clean_data/filing_3  Created 
Directory  /clean_data/filing_4  Created 
Directory  /clean_data/filing_5  Created 
Directory  /clean_data/filing_6  Created 
Directory  /clean_data/filing_7  Created 
Directory  /clean_data/filing_8  Created 
Directory  /clean_data/filing_9  Created 
Directory  /clean_data/filing_10  Created 
Directory  /clean_data/filing_11  Created 
Directory  /clean_data/filing_12  Created 
Directory  /clean_data/filing_13  Created 
Directory  /clean_data/filing_14  Created 
Directory  /clean_data/filing_15  Created 
Directory  /clean_data/filing_16  Created 
Directory  /clean_data/filing_17  Created 
Directory  /clean_data/filing_18  Created 
Directory  /clean_data/filing_19  Created 
Directory  /clean_data/filing_20  Created 
Directory  /clean_data/filing_21  Created 
Directory  /clean_data/filing_22  Created 
Directory  /cle

In [14]:
#BLK file 20 link in <p id=" "
#visa file 2015 have link from item 1-14 but probably forgot to add link to < a item 15 or < a Financial Statement Schedules
#visa file 2015 add link to the page

In [15]:
#remove all xml tag and save it 
"""
remove_xml(soup1)
text_file = open("sec_edgar_filings/clean_data/sample_10k.txt", "w")
text_file.write(str(soup1))
text_file.close()
"""
#load html soup and prettify
"""
soup=load_html("sec_edgar_filings/clean_data/sample_10k.txt")
soup.prettify()
"""
#get table of content links
"""
links = get_table_of_content_links1(soup)
print_list(links)
print(len(links))
"""
#get all sections in 10k
"""
sections=list()
get_all_sections(sections,links)
print(sections[21])
"""
#write sections to a txt file
"""
text_file = open("sample_KG.txt", "w")
text_file.write(sections[2])
text_file.close()
"""

'\ntext_file = open("sample_KG.txt", "w")\ntext_file.write(sections[2])\ntext_file.close()\n'

In [16]:
"""
MSFT_1_path='sec_edgar_filings/MSFT/10-K/0001193125-15-272806.txt'
MSFT_2_path="sec_edgar_filings/MSFT/10-K/0001564590-18-019062.txt"
APPL_1_path='sec_edgar_filings/AAPL/10-K/0000320193-19-000119.txt'
APPL_2_path='sec_edgar_filings/AAPL/10-K/0000320193-18-000145.txt'
VISA_1_path='sec_edgar_filings/V/10-K/0001403161-18-000055.txt'
VISA_2_path='sec_edgar_filings/V/10-K/0001403161-19-000050.txt'
GOOG_1_path='sec_edgar_filings/GOOG/10-K/0001652044-19-000004.txt'
GOOG_2_path='sec_edgar_filings/GOOG/10-K/0001652044-20-000008.txt'
BLK_1_path='sec_edgar_filings/BLK/10-K/0001564590-19-005479.txt'
BLK_2_path='sec_edgar_filings/BLK/10-K/0001564590-20-007807.txt'
BA_1_path='sec_edgar_filings/BA/10-K/0000012927-19-000010.txt'
BA_2_path='sec_edgar_filings/BA/10-K/0000012927-20-000014.txt'
FORD_1_path='sec_edgar_filings/F/10-K/0000037996-20-000010.txt'
FORD_2_path='sec_edgar_filings/F/10-K/0000037996-19-000012.txt'

soup=load_html(MSFT_1_path)
"""
#soup_get_text(soup)
#visa, msft, goog, blk work
#, ba_1_path uses <a id="links">
#ford dont work cuz href is at the page number in toc


'\nMSFT_1_path=\'sec_edgar_filings/MSFT/10-K/0001193125-15-272806.txt\'\nMSFT_2_path="sec_edgar_filings/MSFT/10-K/0001564590-18-019062.txt"\nAPPL_1_path=\'sec_edgar_filings/AAPL/10-K/0000320193-19-000119.txt\'\nAPPL_2_path=\'sec_edgar_filings/AAPL/10-K/0000320193-18-000145.txt\'\nVISA_1_path=\'sec_edgar_filings/V/10-K/0001403161-18-000055.txt\'\nVISA_2_path=\'sec_edgar_filings/V/10-K/0001403161-19-000050.txt\'\nGOOG_1_path=\'sec_edgar_filings/GOOG/10-K/0001652044-19-000004.txt\'\nGOOG_2_path=\'sec_edgar_filings/GOOG/10-K/0001652044-20-000008.txt\'\nBLK_1_path=\'sec_edgar_filings/BLK/10-K/0001564590-19-005479.txt\'\nBLK_2_path=\'sec_edgar_filings/BLK/10-K/0001564590-20-007807.txt\'\nBA_1_path=\'sec_edgar_filings/BA/10-K/0000012927-19-000010.txt\'\nBA_2_path=\'sec_edgar_filings/BA/10-K/0000012927-20-000014.txt\'\nFORD_1_path=\'sec_edgar_filings/F/10-K/0000037996-20-000010.txt\'\nFORD_2_path=\'sec_edgar_filings/F/10-K/0000037996-19-000012.txt\'\n\nsoup=load_html(MSFT_1_path)\n'

In [17]:
"""
#get table of content

"identify the mention of Signatures"
tag = soup.find('a',string=re.compile("Signatures"))
#print(tag)
links = list()
for elements in tag.previous_elements:
    if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
        #print(elements)
        links.append(elements.get('href')[1:])
        
links.reverse()
#print_list(links)
"""

'\n#get table of content\n\n"identify the mention of Signatures"\ntag = soup.find(\'a\',string=re.compile("Signatures"))\n#print(tag)\nlinks = list()\nfor elements in tag.previous_elements:\n    if elements.name==\'a\' and elements.has_attr(\'href\') and elements.get(\'href\')[0]==\'#\':\n        #print(elements)\n        links.append(elements.get(\'href\')[1:])\n        \nlinks.reverse()\n#print_list(links)\n'

In [18]:
#load the html
"""
soup = BeautifulSoup(open("/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/MSFT/10-K/0001564590-19-027952.txt"), 'html.parser')
#print(soup.prettify())
print(soup.get_text())
"""



'\nsoup = BeautifulSoup(open("/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/MSFT/10-K/0001564590-19-027952.txt"), \'html.parser\')\n#print(soup.prettify())\nprint(soup.get_text())\n'

In [19]:
"""
#get all href from the table of content 
Links = list()
for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
    if link.get('href')!=None and link.get('href')[0]=='#':
        Links.append(link.get('href')[1:])
for link in Links:
    print(link)
        
    #remove the '#' and store them in a list
"""

"\n#get all href from the table of content \nLinks = list()\nfor link in soup.find_all('a'):\n    #link.get('href')\n    #print(link.get('href'))\n    #print(link.get('name'))\n    if link.get('href')!=None and link.get('href')[0]=='#':\n        Links.append(link.get('href')[1:])\nfor link in Links:\n    print(link)\n        \n    #remove the '#' and store them in a list\n"

In [20]:
#get section 
#example get section between Item 1 and Item 1A

#Problem also get the sentences
"""
Sections=""
for link in soup.find_all('a'):
    if link.has_attr('name') and link['name']=='ITEM_1_BUSINESS':
        for elements in link.next_elements:
            if elements.name=='a' and elements.has_attr('name') and elements['name']=='ITEM_1B_UNRESOLVED_STAFF_COMMENTS':
                break
                
            
            if str(elements)[0] != '<':
                Sections = Sections + str(elements)
                print(repr(elements))
            #Sections = Sections + str(elements)
            
        break
"""


'\nSections=""\nfor link in soup.find_all(\'a\'):\n    if link.has_attr(\'name\') and link[\'name\']==\'ITEM_1_BUSINESS\':\n        for elements in link.next_elements:\n            if elements.name==\'a\' and elements.has_attr(\'name\') and elements[\'name\']==\'ITEM_1B_UNRESOLVED_STAFF_COMMENTS\':\n                break\n                \n            \n            if str(elements)[0] != \'<\':\n                Sections = Sections + str(elements)\n                print(repr(elements))\n            #Sections = Sections + str(elements)\n            \n        break\n'